In [1]:
#Importing the necessary libraries for the script
import geemap
import ee
from pathlib import Path
import geopandas as gpd
from functions import *

In [2]:
#Authenticating the Google Earth Engine API
ee.Authenticate()
ee.Initialize()
geemap.ee_initialize()

In [3]:
#Setting the Out Directory if not exists creating it
out_dir = Path('../Data')
out_dir.mkdir(exist_ok=True)

#Shapefile directory of the area of interest
shp_dir = Path('../Shapefile/08070201.shp')

In [4]:
studyarea = get_eesupported_roi(shp_dir)

#Earth Engine Collection
dataset_landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterDate('2022-05-01', '2022-07-31')
dataset_landsat9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate('2022-05-01', '2022-07-31')
dataset_sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2022-05-01', '2022-07-31')

In [5]:
#Downloading the image of Landsat 8
landsat_8 = dataset_landsat8.median().clip(studyarea)
save_data(out_dir, landsat_8, 'landsat_8.tif', studyarea)

#Downloading the image of Landsat 9
landsat_9 = dataset_landsat9.median().clip(studyarea)
save_data(out_dir, landsat_9, 'landsat_9.tif', studyarea)

#Downloading the image of Sentinel 2
sentinel_2 = dataset_sentinel2.median().clip(studyarea)
save_data(out_dir, sentinel_2, 'sentinel_2.tif', studyarea)

landsat_9.tif: |          | 0.00/169M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None
